# 17.3 Data Cleaning 2: Missing Values
1. Determine all the variable types and find the fraction of the missing values for each variable.
2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?
3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.
4. This time, fill in the missing values using interpolation (extrapolation).
5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
                    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation', con=engine)

engine.dispose()
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


### 1. The variable types are as follows:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

### Percentage of missing values
Note that the average scores is missing a huge chunk of data at 64% while the rest of the data is between 8~18%.

In [4]:
# return list of percentage of items that are NAN
df.isna().sum() / df.isna().count()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
OTHER_EXPENDITURE               0.176273
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
GRADES_ALL_G                    0.115952
AVG_MATH_4_SCORE                0.640751
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_READING_8_SC

## Central Tendency
Looking at the data the mean is skewed higher than the median of the data. Meaning that there might be some clear outliers in the data. Note that in this assignment the outliers will not be explored. As such the median will be used for the fills below. 

In [5]:
round(df.describe(),2)

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.00,1229.00,1280.00,1280.00,1280.00,1280.00,1280.00,1280.00,1280.00,1229.00,...,1361.00,1361.00,1361.00,1361.00,1361.00,1319.00,536.00,532.00,533.00,498.00
mean,2004.43,915930.82,9092081.86,766372.34,4216552.95,4109156.57,9196680.87,4762966.37,2680330.68,429204.62,...,64538.93,64271.06,54268.92,519214.04,247071.35,802441.51,234.77,278.41,218.87,263.66
std,7.39,1065280.42,11745187.40,1145241.59,5543072.00,5482970.61,11978126.90,6293003.72,3353348.56,534069.32,...,80293.39,78910.55,67889.28,643638.43,307705.63,970703.07,10.22,10.16,7.77,6.80
min,1992.00,43866.00,465650.00,31020.00,0.00,22093.00,481665.00,265549.00,139963.00,11541.00,...,633.00,437.00,311.00,4878.00,1808.00,7254.00,187.13,232.83,178.56,236.38
25%,1998.00,258314.00,2186305.25,189354.00,1153097.25,715834.50,2165404.50,1168032.00,635790.00,102831.00,...,13739.00,13342.00,10919.00,110626.00,51471.00,181065.00,229.69,272.76,214.66,259.53
50%,2004.00,648313.00,5079546.00,403376.50,2537073.50,2055780.50,5234505.50,2657452.00,1525405.50,271596.00,...,43272.00,43339.00,36735.00,345775.00,164260.00,550342.00,237.24,280.62,220.42,265.01
75%,2011.00,1014528.00,10859847.75,828966.00,5080939.25,4768680.00,10745191.00,5568028.00,3229650.75,518600.00,...,75481.00,76566.00,67460.00,611905.00,290502.00,928275.50,242.00,285.35,224.00,268.19
max,2017.00,6307022.00,89217262.00,9990221.00,50904567.00,36105265.00,85320133.00,43964520.00,26058021.00,3995951.00,...,493415.00,500143.00,498403.00,3929869.00,2013687.00,5944746.00,253.42,300.57,236.77,280.50


### 2. 
The median for the population was used to fill the entire dataframe. The issue with this is that the data in the 1992 year is higher than the year 1990s years. This fill will skew the data to a higher amount. The benefit of this fill is that it removes all the NAN items. 

In [6]:
# create list of all columns with NAN items
col_list = ['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 
            'LOCAL_REVENUE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 
            'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 
            'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G',
            'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G','GRADES_9_12_G', 
            'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 
            'AVG_READING_4_SCORE','AVG_READING_8_SCORE']

# fill list by mean
df_fill_1 = df.copy()
df_fill_1[col_list] = df_fill_1[col_list].apply(
    lambda x: x.fillna(x.mean()))
df_fill_1.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,915930.820993,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,263.661132
1,1992_ALASKA,ALASKA,1992,915930.820993,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,234.768293,278.414711,218.866154,258.859712
2,1992_ARIZONA,ARIZONA,1992,915930.820993,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,915930.820993,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,915930.820993,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,263.661132


In [7]:
# list of mean values after the fill
round(df_fill_1.groupby('YEAR').mean(), 2)

,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,,,,,,,,,,,,,,,,,,
1992,915930.82,4976274.88,338935.17,2278335.76,2359003.94,5059224.06,2660158.14,1487129.49,429204.62,449450.79,...,60724.88,56872.36,44038.77,485748.41,207994.39,729438.48,222.50,269.37,214.47,263.61
1993,812221.88,5154768.99,349492.96,2388518.85,2416757.18,5181889.51,2738871.57,1532411.04,264392.81,453654.67,...,61044.32,59019.88,43930.09,491633.55,212404.46,740454.80,234.77,278.41,218.87,263.66
1994,811611.36,5289481.83,366433.07,2403422.01,2519626.75,5333004.88,2816873.10,1584344.10,277763.84,448669.08,...,62184.75,59953.29,45072.16,496646.46,216987.84,751072.77,234.77,278.41,218.87,263.66
1995,864058.56,5798894.97,391376.03,2703390.50,2704128.44,5853843.56,3094656.02,1711285.88,307278.22,514606.04,...,62258.82,60907.04,45061.21,502088.48,222061.82,762232.98,234.77,278.41,218.87,263.66
1996,878351.68,6096583.81,407067.56,2893007.43,2796508.81,6191589.20,3254356.44,1785961.49,316746.50,582250.38,...,61709.25,60714.89,46054.25,502958.63,225201.28,766193.04,224.51,272.49,215.86,262.16
1997,891279.86,6428732.46,426925.91,3074486.26,2927320.29,6509496.58,3405293.04,1858808.95,318319.96,664048.50,...,62649.98,60906.00,47575.40,507648.19,228669.68,774821.02,234.77,278.41,218.87,263.66
1998,899732.37,6816816.31,468180.09,3270588.53,3078047.69,6909682.07,3588928.11,1971284.51,332261.95,727464.66,...,63126.66,61054.21,47627.31,504449.67,227326.98,770974.36,234.77,278.41,218.87,263.66
1999,906569.86,7244379.08,514322.88,3502325.06,3227731.14,7356743.66,3786375.03,2091928.52,347543.08,808515.04,...,64766.90,61405.54,48799.91,508278.70,230884.57,778769.79,234.77,278.41,218.87,263.66
2000,911190.31,7666760.59,555987.64,3727520.16,3383252.65,7792387.39,4002331.85,2210088.80,362764.86,879261.04,...,64045.98,61031.15,48223.34,504013.63,229663.10,785336.65,227.84,274.76,217.93,262.95


In [8]:
# check to see no items are NAN
df_fill_1.isna().sum() / df_fill_1.isna().count()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

## 3. 
For question 3 we will fill the missing data based on the year enrolled. The big issue with this method is that if all the variables in a year is NAN then it will return NAN. The benefit is that the median is more accurate than step 2 above. 

In [9]:
df_fill_2 = df.copy()

# Note that this will apply the fill by the year group and keep the data disaggreagated
df_fill_2 = df_fill_2.groupby('YEAR', as_index=False).apply(
    lambda x: x.fillna(x.mean())).reset_index()
df_fill_2.head()

,level_0,level_1,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,0,0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,263.307067
1,0,1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,218.410116,266.360319,212.712256,258.859712
2,0,2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,0,3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,0,4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,263.307067


In [10]:
# list shows there are still NAN values in 1992 and 2017
df_fill_2.groupby('YEAR').sum(min_count=1)


,level_0,level_1,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,,,,,,,,,,,,,,,,,,
1992,0,8405,NaN,2.560748e+08,1.663366e+07,1.169456e+08,1.224955e+08,2.606011e+08,1.374240e+08,7.672834e+07,...,3.400593e+06,3.184852e+06,2.466171e+06,2.720191e+07,1.164769e+07,4.069714e+07,12230.966510,14916.177865,11911.886312,14745.195745
1993,56,10966,4.491504e+07,2.670504e+08,1.728286e+07,1.237208e+08,1.260468e+08,2.681438e+08,1.422641e+08,7.951271e+07,...,3.418482e+06,3.305113e+06,2.460085e+06,2.753148e+07,1.189465e+07,4.133690e+07,NaN,NaN,NaN,NaN
1994,112,13458,4.487750e+07,2.753340e+08,1.832451e+07,1.246372e+08,1.323723e+08,2.774359e+08,1.470605e+08,8.270609e+07,...,3.482346e+06,3.357384e+06,2.524041e+06,2.781220e+07,1.215132e+07,4.195353e+07,NaN,NaN,NaN,NaN
1995,168,15992,4.810249e+07,3.066579e+08,1.985825e+07,1.430823e+08,1.437173e+08,3.094624e+08,1.641414e+08,9.051176e+07,...,3.486494e+06,3.410794e+06,2.523428e+06,2.811696e+07,1.243546e+07,4.260165e+07,NaN,NaN,NaN,NaN
1996,228,20505,4.981404e+07,3.274178e+08,2.079340e+07,1.560259e+08,1.505985e+08,3.327688e+08,1.753818e+08,9.580227e+07,...,3.517427e+06,3.460749e+06,2.625092e+06,2.866864e+07,1.283647e+07,4.359787e+07,12624.332008,15400.404101,12231.461807,14900.489539
1997,285,22545,5.063765e+07,3.485776e+08,2.205849e+07,1.675872e+08,1.589320e+08,3.530214e+08,1.849973e+08,1.004431e+08,...,3.571049e+06,3.471642e+06,2.711798e+06,2.893595e+07,1.303417e+07,4.410755e+07,NaN,NaN,NaN,NaN
1998,348,26595,5.205553e+07,3.772624e+08,2.478060e+07,1.821635e+08,1.703183e+08,3.825553e+08,1.988116e+08,1.086899e+08,...,3.661346e+06,3.541144e+06,2.762384e+06,2.925808e+07,1.318496e+07,4.465133e+07,NaN,NaN,NaN,NaN
1999,413,31103,5.340099e+07,4.103181e+08,2.801236e+07,2.000271e+08,1.822786e+08,4.170194e+08,2.143579e+08,1.179782e+08,...,3.821479e+06,3.620012e+06,2.873632e+06,2.997732e+07,1.360572e+07,4.587260e+07,NaN,NaN,NaN,NaN
2000,472,33997,5.371636e+07,4.391477e+08,3.085618e+07,2.153977e+08,1.928937e+08,4.467543e+08,2.290980e+08,1.260432e+08,...,3.778713e+06,3.600838e+06,2.845177e+06,2.973680e+07,1.355012e+07,4.628080e+07,13263.408011,16108.687670,12839.143955,15498.550239


In [11]:
# percentage of list that is NAN
df_fill_2.isna().sum() / df_fill_2.isna().count()

level_0                         0.000000
level_1                         0.000000
PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.071716
TOTAL_REVENUE                   0.034182
FEDERAL_REVENUE                 0.034182
STATE_REVENUE                   0.034182
LOCAL_REVENUE                   0.034182
TOTAL_EXPENDITURE               0.034182
INSTRUCTION_EXPENDITURE         0.034182
SUPPORT_SERVICES_EXPENDITURE    0.034182
OTHER_EXPENDITURE               0.071716
CAPITAL_OUTLAY_EXPENDITURE      0.034182
GRADES_PK_G                     0.068365
GRADES_KG_G                     0.068365
GRADES_4_G                      0.068365
GRADES_8_G                      0.068365
GRADES_12_G                     0.068365
GRADES_1_8_G                    0.068365
GRADES_9_12_G                   0.068365
GRADES_ALL_G                    0.068365
AVG_MATH_4_SCORE                0.576408
AVG_MATH_8_SCORE

## 4.
Interpolate populated all the NAN values unlike step 3. One issue could be the outliers skewing the data. 

In [12]:
df_fill_3 = df.copy()

# create an interpolate that fill all values
df_fill_3[col_list] = df_fill_3[col_list].apply(
    lambda x: x.interpolate(method='linear', limit_direction='forward'))
df_fill_3[col_list] = df_fill_3[col_list].apply(
    lambda x: x.interpolate(method='linear', limit_direction='backward'))
df_fill_3[col_list] = df_fill_3[col_list].apply(
    lambda x: x.interpolate(method='nearest'))
round(df_fill_3, 2).head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,89711.0,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.33,252.19,207.96,258.86
1,1992_ALASKA,ALASKA,1992,89711.0,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,211.79,258.78,207.09,258.86
2,1992_ARIZONA,ARIZONA,1992,89711.0,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.25,265.37,206.21,262.17
3,1992_ARKANSAS,ARKANSAS,1992,89711.0,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.21,256.31,208.63,264.62
4,1992_CALIFORNIA,CALIFORNIA,1992,89711.0,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.40,260.89,196.76,265.52


In [13]:
# cheack to see how many nan remain
df_fill_3.isna().sum() / df_fill_3.isna().count()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

## 5. 
For this question all the above items will be compared. The folllowing graphs compares each of the fills above. What was interesting is that the interpolated items had a higher mean than the items that filled in questions 2 and 3. Whereas the median was clearly higher for all the items. It was surprising that overall the data did not change drastically. Out of all the choses the filled values by year was more accurate but had large NAN populations, followed by the interpolated items. 

In [14]:
# create describe list for each fill to compare with original dataframe
for col in col_list:
    print("Statistics for columns: {}".format(col))
    print(pd.concat([df[col], df_fill_1[col],
                     df_fill_2[col], df_fill_3[col]], axis=1).describe())

Statistics for columns: ENROLL
             ENROLL        ENROLL        ENROLL        ENROLL
count  1.229000e+03  1.492000e+03  1.385000e+03  1.492000e+03
mean   9.159308e+05  9.159308e+05  9.161654e+05  9.838759e+05
std    1.065280e+06  9.667725e+05  1.003532e+06  1.006773e+06
min    4.386600e+04  4.386600e+04  4.386600e+04  4.386600e+04
25%    2.583140e+05  3.150940e+05  2.867590e+05  2.821195e+05
50%    6.483130e+05  8.204140e+05  7.374010e+05  7.373935e+05
75%    1.014528e+06  9.211780e+05  9.624880e+05  1.595024e+06
max    6.307022e+06  6.307022e+06  6.307022e+06  6.307022e+06
Statistics for columns: TOTAL_REVENUE
       TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE
count   1.280000e+03   1.492000e+03   1.441000e+03   1.492000e+03
mean    9.092082e+06   9.092082e+06   9.084108e+06   1.111164e+07
std     1.174519e+07   1.087818e+07   1.110204e+07   1.200155e+07
min     4.656500e+05   4.656500e+05   4.656500e+05   4.656500e+05
25%     2.186305e+06   2.546261e+06   2.425

          GRADES_8_G     GRADES_8_G     GRADES_8_G     GRADES_8_G
count    1361.000000    1492.000000    1390.000000    1492.000000
mean    64271.057311   64271.057311   64331.744106   59511.380027
std     78910.545776   75364.301721   78083.976028   77123.573668
min       437.000000     437.000000     437.000000     437.000000
25%     13342.000000   15682.750000   13698.250000   10252.000000
50%     43339.000000   49571.000000   44756.500000   38114.500000
75%     76566.000000   72131.000000   75016.250000   72598.250000
max    500143.000000  500143.000000  500143.000000  500143.000000
Statistics for columns: GRADES_12_G
        GRADES_12_G    GRADES_12_G    GRADES_12_G    GRADES_12_G
count    1361.00000    1492.000000    1390.000000    1492.000000
mean    54268.92432   54268.924320   54418.075582   50207.239276
std     67889.27623   64838.328605   67187.337112   66309.867155
min       311.00000     311.000000     311.000000     311.000000
25%     10919.00000   13437.000000   11161.75